In [313]:
import torch
import math


def EularAngleToMatrix(eularAngle):
    sinEular = eularAngle.sin()
    cosEular = eularAngle.cos()
    matrix = torch.eye(4)

    matrix[0][0] = sinEular[0] * sinEular[1] * sinEular[2] + cosEular[0] * cosEular[2]
    matrix[0][1] = cosEular[1] * sinEular[2] 
    matrix[0][2] = cosEular[0] * sinEular[1] * sinEular[2] - sinEular[0] * cosEular[2]
    matrix[1][0] = sinEular[0] * sinEular[1] * cosEular[2] - cosEular[0] * sinEular[2]
    matrix[1][1] = cosEular[1] * cosEular[2] 
    matrix[1][2] = cosEular[0] * sinEular[1] * cosEular[2] + sinEular[0] * sinEular[2]
    matrix[2][0] = sinEular[0] * cosEular[1]
    matrix[2][1] = -sinEular[1] 
    matrix[2][2] = cosEular[0] * cosEular[1]
    return matrix

def MatrixToEularAngle(matrix):
    eularAngle = torch.zeros(3)

    sinPitch = -matrix[2][1]
    if sinPitch <= -1.0:
        eularAngle[1] = torch.pi * -0.5
    elif sinPitch > 1:
        eularAngle[1] = torch.pi * 0.5
    else:
        eularAngle[1] = torch.asin(sinPitch)

    if torch.abs(sinPitch) >= 0.999999:
        eularAngle[0] = torch.atan2(-matrix[0][2], matrix[0][0])
        eularAngle[2] = 0
    else:
        eularAngle[0] = torch.atan2(matrix[2][0], matrix[2][2])
        eularAngle[2] = torch.atan2(matrix[0][1], matrix[1][1])
        if eularAngle[2] < torch.pi * -0.5 or torch.pi * 0.5 < eularAngle[2]:
            eularAngle[1] = torch.pi - eularAngle[1]
            eularAngle[0] = torch.atan2(-matrix[2][0], -matrix[2][2])
            eularAngle[2] = torch.atan2(-matrix[0][1], -matrix[1][1])
    return eularAngle

viewport = torch.tensor([2648.0,1592.0])
aspect = viewport[0]/viewport[1]

def fovyToScale(degree):
    fovy = degree*math.pi/180.0
    yScale = 1.0/math.tan(fovy*0.5)
    xScale = yScale / aspect
    return torch.tensor([xScale, yScale]) 


def PixelToProjection(xy):
    wh = viewport
    p = xy / wh * torch.tensor([2.0, -2.0]) +  torch.tensor([-1.0, 1.0])
    return p

def ProjectionToPixel(xy):
    wh = viewport
    p = (xy - torch.tensor([-1.0, 1.0])) / torch.tensor([2.0, -2.0]) * wh
    return p

offset = torch.tensor([
    [288.0, 148.0]
    ])

pixels1 = torch.tensor([
    [602.0, 1183.0],
    [2489.0, 656.0],
    [1596.0, 1383.0],
])

pixels2 = torch.tensor([
    [646.0, 965.0],
    [2103.0, 559.0],
    [1433.0, 1109.0],
])

pixels1 = pixels1 - offset
pixels2 = pixels2 - offset
pixels1,pixels2

proj1 = fovyToScale(35.0)
p1 = PixelToProjection(pixels1)
v = torch.cat([p1 / proj1, torch.ones(p1.shape[0],1)], dim=1)

'''rotation2 = torch.eye(4, requires_grad=True)
proj2 = torch.ones(2, requires_grad=True)
optimizer = torch.optim.Adam([rotation2, proj2], lr = 1e-2)
epochs = 5000
for epoch in range(epochs):
    v2 = torch.matmul(v, rotation2[0:3,0:3])
    p2 = v2[:,0:2] * proj2
    yhat = ProjectionToPixel(p2) 
    loss1 = torch.nn.functional.mse_loss(yhat, pixels2).mean()
    loss2 = torch.nn.functional.mse_loss(proj2[1]/proj2[0], aspect).mean()*10
    #loss2 = torch.nn.functional.mse_loss(v.pow(2).sum(dim=1), v2.pow(2).sum(dim=1)).mean()
    loss3 = torch.dot(rotation2[0],rotation2[1]).pow(2)*1
    loss4 = torch.dot(rotation2[0],rotation2[2]).pow(2)*1
    loss5 = torch.dot(rotation2[1],rotation2[2]).pow(2)*1
    loss6 = torch.nn.functional.mse_loss(rotation2.pow(2).sum(dim=1), torch.ones(4)).mean()*1000
    
    if epoch == epochs-1:
        print("----", loss1, loss3, loss4, loss5, loss6, "\n", yhat, "\n", pixels2)
    loss = loss1 + loss2 + loss3 + loss4 + loss5 + loss6
    #print(norm0, loss)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch*10 % epochs ==0:
        print(loss)

print(torch.atan(1.0/proj2)*2/math.pi*180, MatrixToEularAngle(rotation2)*180/math.pi)
eularAngle2 = MatrixToEularAngle(rotation2)
rotation3 = EularAngleToMatrix(eularAngle2)
v2 = torch.matmul(v, rotation3[0:3,0:3])
p2 = v2[:,0:2] * proj2
yhat = ProjectionToPixel(p2) 
'''


eularAngle2 = torch.zeros(3, requires_grad=True)
proj2 = torch.ones(2, requires_grad=True)
optimizer = torch.optim.Adam([eularAngle2, proj2], lr = 1e-2)
epochs = 1000
for epoch in range(epochs):
    rotation2 = EularAngleToMatrix(eularAngle2)
    v2 = torch.matmul(v, rotation2[0:3,0:3])
    p2 = v2[:,0:2] * proj2
    yhat = ProjectionToPixel(p2) 
    loss1 = torch.nn.functional.mse_loss(yhat, pixels2).mean()
    loss2 = torch.nn.functional.mse_loss(proj2[1]/proj2[0], aspect).mean()*10
    
    if epoch == epochs-1:
        print("----", loss1, loss2, "\n", yhat, "\n", pixels2)
    loss = loss1 + loss2
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch*10 % epochs ==0:
        print(loss)

torch.atan(1.0/proj2)*2/math.pi*180, eularAngle2*180/math.pi


tensor(51779.9961, grad_fn=<AddBackward0>)
tensor(923.5286, grad_fn=<AddBackward0>)
tensor(166.7108, grad_fn=<AddBackward0>)
tensor(47.3521, grad_fn=<AddBackward0>)
tensor(36.7062, grad_fn=<AddBackward0>)
tensor(36.1802, grad_fn=<AddBackward0>)
tensor(36.1660, grad_fn=<AddBackward0>)
tensor(36.1659, grad_fn=<AddBackward0>)
tensor(36.1658, grad_fn=<AddBackward0>)
tensor(36.1656, grad_fn=<AddBackward0>)
---- tensor(36.1078, grad_fn=<MeanBackward0>) tensor(0.0577, grad_fn=<MulBackward0>) 
 tensor([[ 363.2495,  821.5812],
        [1820.9304,  409.9081],
        [1134.0031,  957.7125]], grad_fn=<MulBackward0>) 
 tensor([[ 358.,  817.],
        [1815.,  411.],
        [1145.,  961.]])


(tensor([68.0016, 46.0454], grad_fn=<MulBackward0>),
 tensor([-5.2899, -4.9899,  0.9112], grad_fn=<DivBackward0>))

In [293]:
torch.atan(1.0/proj2)*2/math.pi*180, MatrixToEularAngle(rotation2)*180/math.pi

(tensor([68.2349, 44.9136], grad_fn=<MulBackward0>),
 tensor([-5.2932, -4.9270,  0.4571], grad_fn=<DivBackward0>))

In [294]:
rotation2,rotation2[0]
#torch.dot(rotation2[1],rotation2[2]).pow(2)

proj2[1]/proj2[0], aspect

(tensor(1.6391, grad_fn=<DivBackward0>), tensor(1.6633))